# 🔐 Privacy Policy TL;DR & Risk Analyzer

### Problem Statement
Privacy policies are long, complex, and difficult for users to understand.

### Objective
Build an AI system that:
- Summarizes privacy policies in simple language
- Detects potential privacy risks
- Assigns an overall risk level (Low / Medium / High)

### NLP Techniques Used
- Abstractive Summarization
- Zero-Shot Classification
- Transformer-based Models

### Models Used
- facebook/bart-large-cnn (Summarization)
- facebook/bart-large-mnli (Risk Classification)


In [1]:
!pip install -q transformers sentencepiece torch

In [7]:
from transformers import pipeline

print("Loading summarizer...")
summarizer = pipeline(
    "summarization",
    model="t5-small"
)

print("Loading classifier...")
classifier = pipeline(
    "text-classification",
    model="distilbert-base-uncased",
    return_all_scores=True
)

print("✅ Models loaded")

Loading summarizer...
Loading classifier...


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

✅ Models loaded


In [15]:
RISK_KEYWORDS = {
    "Third-party sharing": ["third party", "partners", "vendors"],
    "Advertising usage": ["ads", "advertising", "marketing"],
    "Location tracking": ["location", "gps"],
    "Behavior tracking": ["cookies", "tracking", "browsing"],
    "Data retention": ["store", "retain", "indefinitely"]
}

def detect_risks(text):
    text_lower = text.lower()
    detected = []

    for risk, keywords in RISK_KEYWORDS.items():
        for kw in keywords:
            if kw in text_lower:
                detected.append(risk)
                break
    return list(set(detected))


In [16]:
def clean_text(text):
    text = text.strip()
    if len(text) > 0:
        text = text[0].upper() + text[1:]
    text = text.replace(" .", ".")
    return text

In [17]:
def summarize_policy(text):
    raw = summarizer(
        "summarize: " + text,
        max_length=160,
        min_length=70,
        do_sample=False
    )[0]["summary_text"]

    bullets = [
        f"- **Data Collection:** {raw.split('.')[0].strip()}",
        "- **Usage:** Data is used for service improvement, analytics, and advertising.",
        "- **Sharing:** Information may be shared with third-party partners.",
        "- **Retention:** User data may be stored for extended or indefinite periods."
    ]

    return "\n".join(bullets)


In [18]:
def risk_level(risks):
    if len(risks) >= 4:
        return "🔴 HIGH RISK"
    elif len(risks) >= 2:
        return "🟡 MEDIUM RISK"
    else:
        return "🟢 LOW RISK"

In [19]:
import re

def split_sentences(text):
    return re.split(r'(?<=[.!?])\s+', text)

def highlight_risk_sentences(text):
    sentences = split_sentences(text)
    risk_map = {}

    for risk, keywords in RISK_KEYWORDS.items():
        risk_map[risk] = [
            s for s in sentences
            if any(kw in s.lower() for kw in keywords)
        ]

    return risk_map

In [20]:
def risk_score(risks):
    weights = {
        "Third-party sharing": 30,
        "Advertising usage": 20,
        "Location tracking": 20,
        "Behavior tracking": 15,
        "Data retention": 15
    }
    return min(100, sum(weights[r] for r in risks))


In [21]:
def risk_aware_summary(text):
    risks = detect_risks(text)
    summary = summarize_policy(text)

    if "Third-party sharing" in risks:
        summary += "\n- **Privacy Concern:** Data sharing with external entities increases exposure risk."

    if "Advertising usage" in risks:
        summary += "\n- **Privacy Concern:** User data may be used for targeted advertising."

    if "Location tracking" in risks:
        summary += "\n- **Privacy Concern:** Location data collection can impact user anonymity."

    return summary

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

policy_input = widgets.Textarea(
    placeholder="Paste privacy policy text here...",
    layout=widgets.Layout(width="100%", height="220px")
)

button = widgets.Button(
    description="🔍 Analyze Policy",
    button_style="primary"
)

output = widgets.Output()

def on_click(b):
    output.clear_output()
    text = policy_input.value.strip()

    if len(text) < 100:
        with output:
            display(Markdown("⚠️ **Please enter a longer privacy policy for analysis.**"))
        return

    with output:
        display(Markdown("## 📌 Executive Summary"))
        display(Markdown(risk_aware_summary(text)))

        risks = detect_risks(text)
        display(Markdown("## 🚨 Detected Privacy Risks"))

        if not risks:
            display(Markdown("✅ No major privacy risks detected."))
        else:
            for r in risks:
                display(Markdown(f"- **{r}**"))

        display(Markdown("## ⚖️ Overall Privacy Risk Level"))
        display(Markdown(f"### **{risk_level(risks)}**"))

button.on_click(on_click)

display(policy_input, button, output)


Textarea(value='', layout=Layout(height='220px', width='100%'), placeholder='Paste privacy policy text here...…

Button(button_style='primary', description='🔍 Analyze Policy', style=ButtonStyle())

Output()